In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from pprint import pprint
from datetime import datetime
import re

executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)

In [ ]:
test_url_df = pd.read_csv("test_playlist_urls.csv")

test_url_list = test_url_df['0'].tolist()

In [19]:
def dateFixer(str):
    if str == '':
        return ['', '', '', '']
    else:
        date_list = []
        str = re.sub(r'[.]', ':', str)
        parts = str.split()
        parts[2] = parts[2].strip("stndrh")
        str = " ".join(parts)
        date = re.search(r'^(.*?)\d\d\d\d', str).group(0)
        start_time = re.search(r'.+?(?=–)', str).group(0)
        end_time = date + " " + re.search(r'(?<=–).*', str).group(0)
        start_dt_obj = datetime.strptime(start_time, '%a %b %d %Y %I:%M%p')
        end_dt_obj = datetime.strptime(end_time, '%a %b %d %Y %I:%M%p')
        date_list.append(start_time)
        date_list.append(start_dt_obj)
        date_list.append(end_time)
        date_list.append(end_dt_obj)
        return date_list


In [20]:
dateFixer('')

['', '', '', '']

In [7]:
browser.visit('http://kdhx.org/shows/schedule')
browser.is_element_present_by_css("p.plhead", wait_time=1)
html = browser.html
schedule_soup = bs(html, 'html.parser')


In [8]:
weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
kdhx_genres = {}
for day in weekdays:
    schedule = schedule_soup.find('div', class_='day', dow=day).find_all('a')
    program_list = []
    for item in schedule:
        program = item.find('div', class_='show-title').find('span', style='margin-right:5px').get_text().strip()
        program_list.append(program)
        #print(program)
        try:
            program = re.search(r'.+?(?=\s\s)', program).group(0)
        except:
            program = program
        kdhx_genres.setdefault(program, {})
        genre_elements = item.find('div', class_='show-genres').find_all('span')
        genres = []
        for element in genre_elements:
            genres.append(element.get_text())
        kdhx_genres[program] = genres

In [ ]:
for playlist_url in test_url_list:
    print(playlist_url)
    browser.visit(playlist_url)
    browser.is_element_present_by_css("p.plhead", wait_time=1)
    html = browser.html
    playlist_soup = bs(html, 'html.parser')
    

In [36]:
#kdhx_genres['No Time To Tarry Here'] = kdhx_genres.pop('No Time to Tarry Here')
#kdhx_genres['Music from the Hills'] = kdhx_genres.pop('Music From the Hills')
#kdhx_genres['Boogie On Down'] = kdhx_genres.pop('Boogie on Down')
#kdhx_genres['Howzit Bayou'] = kdhx_genres.pop('Howzit Bayou?')
#kdhx_genres['Cure For Pain'] = kdhx_genres.pop('Cure for Pain')
#kdhx_genres["Shake 'em on Down"] = kdhx_genres.pop("Shake 'Em on Down")
#kdhx_genres["Chicken Shack"] = kdhx_genres.pop("Chicken Shack Alley")
kdhx_genres

{'Elevated Rhymestate': ['Hip Hop'],
 'Latin Hemispheres': ['World'],
 'After Hours': ['Electronic'],
 'Sunday Morning Show': [''],
 'Lotsa A Cappella': ['Folk', 'World'],
 'Folks of the World': ['Folk', 'World'],
 'Songwriters Showcase': ['Folk', 'Singer/Songwriter'],
 'Bluegrass Breakdown': ['Bluegrass', 'Folk'],
 'Chicken Shack': ['Blues', 'R&B', 'Soul'],
 "Nothin' but the Blues": ['Blues'],
 'All Soul, No Borders': ['Jazz', 'Soul'],
 'In the Middle of the Night': ['Jazz'],
 "The Lion's Den": ['Rock'],
 'Sonic Space': ['Funk', 'Hip Hop', 'R&B', 'Rock', 'Electronic'],
 'Mystery Train': ['Country', 'Folk', 'Rock'],
 'Radio Rio': ['World'],
 'Dangerous Curves': ['Blues', 'R&B', 'Rock', 'Soul'],
 'Soul Selector': ['Blues', 'Soul'],
 'Music at Work': ['Americana', 'Rock'],
 'loudQUIETloud': ['Rock'],
 'Serious Journalism': ['Rock'],
 'C-Sides': ['Rock'],
 'Universal Default': ['Rock'],
 'The Audio Pharm': ['Americana', 'Folk', 'Rock'],
 'Rocket 88': ['Rock'],
 "Sittin' On Top of the Worl

In [ ]:
playlist_soup.find('div', id='playlisthead').find('p', class_='indent').get_text()


In [ ]:
playlist_soup.find('p', class_='indent').get_text()

In [ ]:
program_name = playlist_soup.find('p', class_='plhead').find('a').get_text()
program_name

In [24]:
test_url = ['https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=17372#here', 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62550#here']

In [28]:
main_df = pd.DataFrame({'program': '', 'start_time': '', 'start_dt_object': '', 'end_time': '',
                        'end_dt_object': '', 'description': '', 'play_time': '', 'artist': '',       
                        'artist_url': '', 'track': '', 'album': '', 'album_url': '', 'label': '', 
                        'label_url': '', 'type': '', 'notes': '', 'play_duration': ''}, index=[x for x in range(0)])
kdhx_dict = {}
for playlist_url in test_url:
    print(playlist_url)
    browser.visit(playlist_url)
    browser.is_element_present_by_css("p.plhead", wait_time=1)
    html = browser.html
    playlist_soup = bs(html, 'html.parser')
    try:
        program_name = playlist_soup.find('p', class_='plhead').find('a').get_text()
    except:
        program_name = ''
    try:
        play_date = playlist_soup.find('p', class_='plheadsub').get_text()
    except:
        play_date = ''
    try:
        dj = playlist_soup.find('div', class_='infoblock').find('p', class_='plhead').find('a').get_text()
    except:
        dj = ''
    kdhx_dict.setdefault(program_name, {}).setdefault(dateFixer(play_date)[0], {}).setdefault(dj, {})
    kdhx_dict[program_name]['genre'] = kdhx_genres[program_name]
    kdhx_dict[program_name]['end time'] = dateFixer(play_date)[2]
    kdhx_dict[program_name]['start_dt_object'] = dateFixer(play_date)[1]
    kdhx_dict[program_name]['end_dt_object'] = dateFixer(play_date)[3]
    try:
        kdhx_dict[program_name]['description'] = playlist_soup.find('div', id='playlisthead').find('p', class_='indent').get_text()
    except:
        kdhx_dict[program_name]['description'] = ""
    play_dict = {}
    playlist_div = playlist_soup.find('div', class_='plblock')
    play_divs = playlist_div.find_all('div', class_='f2row')
    for index, play in enumerate(play_divs):
        try:
            play_time = play.find('p', class_='st').get_text()
        except:
            play_time = ''
        try:
            if index == len(play_divs) - 1:
                time_delta = datetime.strptime(re.search(r'(?<=\d\d\d\d\s).*', dateFixer(play_date)[2]).group(0), '%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
            else:
                time_delta = datetime.strptime(play_divs[index+1].find('p', class_='st').get_text(),'%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
            play_duration = time_delta.total_seconds() / 60
        except:
            play_duration = ''
        kdhx_dict[program_name][dateFixer(play_date)[0]][dj].setdefault(play_time, {})
        play_dict = {'artist': '', 'artist_url': '', 'track': '', 'album': '', 'album_url': '',    
                     'label': '', 'label_url': '', 'type': '', 'notes': '', 'play_duration': ''}
        play_dict['play_duration'] = play_duration        
        try:
            play_dict['artist'] = play.find('span', class_='aw').get_text()
        except:
            play_dict['artist'] = ''
        try:
            play_dict['artist_url'] = "https://spinitron.com/radio/" + play.find('span', class_='aw').find('a')['href']
        except:
            play_dict['artist_url'] = ''
        try:
            play_dict['track'] = play.find('span', class_='sn').get_text()
        except:
            play_dict['track'] = ''
        try:
            play_dict['album'] = play.find('span', class_='dn').get_text()
        except:
            play_dict['album'] = ''
        try:
            play_dict['album_url'] = "https://spinitron.com/radio/" + play.find('span', class_='dn').find('a')['href']
        except:
            play_dict['album_url'] = ''
        try:
            play_dict['label'] = play.find('span', class_='ld').get_text()
        except:
            play_dict['label'] = ''
        try:
            play_dict['label_url'] = "https://spinitron.com/radio/" + play.find('span', class_='ld').find('a')['href']
        except:
            play_dict['label_url'] = ''
        try:
            play_dict['type'] = play.find('span', class_='fg').get_text()
        except:
            play_dict['type'] = ''
        try:
            play_dict['notes'] = play.find('span', class_='so').get_text()
        except:
            play_dict['notes'] = ''
        kdhx_dict[program_name][dateFixer(play_date)[0]][dj][play_time] = play_dict
    for index, play in enumerate(play_divs):
        this_df = pd.DataFrame({'program': '', 'start_time': '', 'start_dt_object': '', 'end_time': '',
                        'end_dt_object': '', 'description': '', 'play_time': '', 'artist': '',       
                        'artist_url': '', 'track': '', 'album': '', 'album_url': '', 'label': '', 
                        'label_url': '', 'type': '', 'notes': '', 'play_duration': ''}, index=[x for x in range(1)])
        try:
            play_time = play.find('p', class_='st').get_text()
        except:
            play_time = ''
        try:
            if index == len(play_divs) - 1:
                time_delta = datetime.strptime(re.search(r'(?<=\d\d\d\d\s).*', dateFixer(play_date)[2]).group(0), '%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
            else:
                time_delta = datetime.strptime(play_divs[index+1].find('p', class_='st').get_text(),'%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
            play_duration = time_delta.total_seconds() / 60
        except:
            play_duration = ''
        for index, row in this_df.iterrows():
            this_df.loc[index, 'program'] = program_name
            this_df.loc[index, 'start_time'] = dateFixer(play_date)[0]
            this_df.loc[index, 'start_dt_object'] = dateFixer(play_date)[1]
            this_df.loc[index, 'end_time'] = dateFixer(play_date)[2]
            this_df.loc[index, 'end_dt_object'] = dateFixer(play_date)[3]
            try:
                this_df.loc[index, 'description'] = playlist_soup.find('div', id='playlisthead').find('p', class_='indent').get_text()
            except:
                this_df.loc[index, 'description'] = ""
            this_df.loc[index, 'play_time'] = play_time
            this_df.loc[index, 'play_duration'] = play_duration
            try:
                this_df.loc[index, 'artist'] = play.find('span', class_='aw').get_text()
            except:
                this_df.loc[index, 'artist'] = ''
            try:
                this_df.loc[index, 'artist_url'] = "https://spinitron.com/radio/" + play.find('span', class_='aw').find('a')['href']
            except:
                this_df.loc[index, 'artist_url'] = ''
            try:
                this_df.loc[index, 'track'] = play.find('span', class_='sn').get_text()
            except:
                this_df.loc[index, 'track'] = ''
            try:
                this_df.loc[index, 'album'] = play.find('span', class_='dn').get_text()
            except:
                this_df.loc[index, 'album'] = ''
            try:
                this_df.loc[index, 'album_url'] = "https://spinitron.com/radio/" + play.find('span', class_='dn').find('a')['href']
            except:
                this_df.loc[index, 'album_url'] = ''
            try:
                this_df.loc[index, 'label'] = play.find('span', class_='ld').get_text()
            except:
                this_df.loc[index, 'label'] = ''
            try:
                this_df.loc[index, 'label_url'] = "https://spinitron.com/radio/" + play.find('span', class_='ld').find('a')['href']
            except:
                this_df.loc[index, 'label_url'] = ''
            try:
                this_df.loc[index, 'type'] = play.find('span', class_='fg').get_text()
            except:
                this_df.loc[index, 'type'] = ''
            try:
                this_df.loc[index, 'notes'] = play.find('span', class_='so').get_text()
            except:
                this_df.loc[index, 'notes'] = ''
        main_df = pd.concat([main_df, this_df])

https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=17372#here
https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62550#here


In [29]:
kdhx_dict

{'Music from the Hills': {'Sun Sep 7 2014 4:00pm': {'John Uhlemann': {'': {'artist': 'Romanyi Rota',
     'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=14541#here',
     'track': '“Phiravelman kalyi phuv”',
     'album': 'Cigány népdalok',
     'album_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=d&dbid=125381#here',
     'label': '(Fonó www.fono.hu 1999)',
     'label_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=l&dbid=4968#here',
     'type': '',
     'notes': '',
     'play_duration': ''}}},
  'genre': ['Folk', 'World'],
  'end time': 'Sun Jan 20 2019 6:00pm',
  'start_dt_object': datetime.datetime(2019, 1, 20, 16, 0),
  'end_dt_object': datetime.datetime(2019, 1, 20, 18, 0),
  'description': 'John Uhlemann brings you 2 hours of strangely beautiful folk and traditional music from Eastern Europe and Scandinavia, with occasional forays into the Middle East and Western Europe.',
  'Su

In [32]:
main_df = main_df.reset_index().drop(columns='index')

In [33]:
main_df

,program,start_time,start_dt_object,end_time,end_dt_object,description,play_time,artist,artist_url,track,album,album_url,label,label_url,type,notes,play_duration
0,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Arto & Antti Järvelä,https://spinitron.com/radio/playlist.php?stati...,“Reinon Knäpsäys”,Os Fera Liluli,https://spinitron.com/radio/playlist.php?stati...,(OArt music 2013),https://spinitron.com/radio/playlist.php?stati...,,—This show was devoted to pieces asked for by ...,
1,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Niekku,https://spinitron.com/radio/playlist.php?stati...,“Polkkaa Ja Polskaa”,Niekku,https://spinitron.com/radio/playlist.php?stati...,(Kansanmusiikki-Instituuti 1993),https://spinitron.com/radio/playlist.php?stati...,,,
2,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Loituma,https://spinitron.com/radio/playlist.php?stati...,“Ievan Polkka (Ieva's Polka)”,Things Of Beauty,https://spinitron.com/radio/playlist.php?stati...,(Northside www.noside.com 2006),https://spinitron.com/radio/playlist.php?stati...,,,
3,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Svitanya,https://spinitron.com/radio/playlist.php?stati...,“Hubava Milka (Bulgaria)”,Our Village,https://spinitron.com/radio/playlist.php?stati...,((self produced) 2012),https://spinitron.com/radio/playlist.php?stati...,,,
4,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Svitanya,https://spinitron.com/radio/playlist.php?stati...,“Prexvrukhna Ptichka (Bulgaria)”,Our Village,https://spinitron.com/radio/playlist.php?stati...,((self produced) 2012),https://spinitron.com/radio/playlist.php?stati...,,,
5,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Téka,https://spinitron.com/radio/playlist.php?stati...,“Vajdaszentiványi Sebesforduló”,Erdélyország Sok Szép Vize,https://spinitron.com/radio/playlist.php?stati...,(Duo Sound 1993),https://spinitron.com/radio/playlist.php?stati...,,,
6,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Kárpátia,https://spinitron.com/radio/playlist.php?stati...,“Keserves és kezesek”,Kárpátia,https://spinitron.com/radio/playlist.php?stati...,(Etnofon 1998),https://spinitron.com/radio/playlist.php?stati...,,,
7,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Orkiestra Św. Mikołaja i kapela Romana Kumlyka,https://spinitron.com/radio/playlist.php?stati...,“Kołomyjki zakarpackie”,Huculskie Muzyki,https://spinitron.com/radio/playlist.php?stati...,((self) 2005),https://spinitron.com/radio/playlist.php?stati...,,,
8,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Nicu Pourvu and ensemble,https://spinitron.com/radio/playlist.php?stati...,“Ciocîrlia”,Panpipes of Rumania - the fantastic virtuosity...,https://spinitron.com/radio/playlist.php?stati...,(Capitol (LP)),https://spinitron.com/radio/playlist.php?stati...,,—This was a challenge from a listener - they a...,
9,Music from the Hills,Sun Sep 7 2014 4:00pm,2014-09-07 16:00:00,Sun Sep 7 2014 6:00pm,2014-09-07 18:00:00,John Uhlemann brings you 2 hours of strangely ...,,Ando Drom,https://spinitron.com/radio/playlist.php?stati...,“Zsa mo /

In [ ]:
url = "https://spinitron.com/radio/playlist.php?station=kdhx&show=schedule&sv=l#here"
browser.is_element_present_by_css("p.plhead", wait_time=1)
browser.visit(url)

html = browser.html
schedule_soup = bs(html, 'html.parser')

schedule_tables = schedule_soup.find_all('tbody')
schedule_tables = schedule_tables[2:8]

program_list = []
for day in schedule_tables:
    day_list = []
    day_list = day.find_all('p')
    for program in day_list:
        program_list.append(program.get_text())

program_url_list = []
for day in schedule_tables:
    day_list = []
    day_list = day.find_all('a')
    for program in day_list:
        program_url_list.append('https://spinitron.com/radio/' + program['href'])
program_url_list = program_url_list[::2]

all_playlist_urls = []
for program_url in program_url_list:
    print(program_url)
    browser.visit(program_url)
    html = browser.html
    program_soup = bs(html, 'html.parser')
    playlist_table = program_soup.find('table', id='pltable')
    playlist_anchors = playlist_table.find_all('a', title='Click for the playlist')
    playlist_url_list = []
    for anchor in playlist_anchors:
        all_playlist_urls.append('https://spinitron.com/radio/' + anchor['href'])
all_playlist_urls = all_playlist_urls[::2]

all_url_df = pd.DataFrame(all_playlist_urls)

In [ ]:
all_url_list = all_url_df[0].tolist()
all_url_list